<a href="https://colab.research.google.com/github/vitao-bolado/Trabalho2_AnaliseDados/blob/main/Trabalho2_AnaliseDados_VitorFaustinoTirabassi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Trabalho 2 - Análise de Desempenho Estudantil**
Desenvolvido por** Vitor Faustino Tirabassi

O objetivo é elaborar uma análise dos fatores que influenciam o desempenho acadêmico dos estudantes

Desafios:

* Avaliar a influência de sono, motivação e atividades extracurriculares.

* Verificar o impacto do nível socioeconômico.

* Identficar padrões de sucesso escolar.






##**Etapa 1 - Importação das bibliotecas necessárias**

Nesta etapa, serão impotadas as bibliotecas
necessárias. Estas bibliotecas fornecem ferramentas para manipulação de dados (pandas, numpy),
visualização (matplotlib, seaborn) e criação e avaliação de modelos preditivos (sklearn).

In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Modelagem de dados
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier

# Avaliação dos modelos
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Configurações gerais
import warnings
warnings.filterwarnings('ignore')

# Conexão com Google Drive
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive"

Mounted at /content/drive


##**Etapa 2 - Carregamento e visualização inicial dos dados**

Nesta etapa, é carregada a base de dados fornecida e observada sua estrutura, permitindo entender quais variáveis estão presentes, seus tipos e se existem problemas
iniciais como dados ausentes ou colunas desnecessárias.

In [ ]:
# Leitura do arquivo CSV
df = pd.read_csv("/content/drive/MyDrive/03_desempenho_estudantil.csv")

# Exibir as primeiras 5 linhas
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


## **Etapa 3 - Análise da estrutura dos dados e tratamento inicial**



**Etapa 3.1 - Verificação da estrutura da base**

Aqui será analisada a dimensão da base de dados (número de registros e variáveis), identificar os tipos de dados presentes
(variáveis categóricas ou numéricas), e verificar se a base está pronta para análise ou necessita de ajustes.

In [ ]:
# Quantidade de registros e variáveis
print(f'A base possui {df.shape[0]} registros e {df.shape[1]} variáveis.')

# Tipos de variáveis
print("\nTipos de dados por coluna:\n")
print(df.dtypes)

A base possui 6607 registros e 20 variáveis.

Tipos de dados por coluna:

Hours_Studied                  int64
Attendance                     int64
Parental_Involvement          object
Access_to_Resources           object
Extracurricular_Activities    object
Sleep_Hours                    int64
Previous_Scores                int64
Motivation_Level              object
Internet_Access               object
Tutoring_Sessions              int64
Family_Income                 object
Teacher_Quality               object
School_Type                   object
Peer_Influence                object
Physical_Activity              int64
Learning_Disabilities         object
Parental_Education_Level      object
Distance_from_Home            object
Gender                        object
Exam_Score                     int64
dtype: object
